In [1]:
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Activation,AveragePooling1D
import numpy as np
import pandas as pd
from keras import initializers

Using TensorFlow backend.


In [2]:
# construct the model used for classifier
# In our case the input shape is (4096,1) and is already included.

def classifier():  
    
    model = Sequential()
    # 2 "convo+pooling" layers 
    model.add(Conv2D(filters=5, kernel_size=(3,3), strides=1, input_shape=(4096,1)))
    model.add(MaxPooling1D(pool_size=(2,2), strides=4))
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=10, kernel_size=(3,3), strides=1, input_shape=(4096,1)))
    model.add(MaxPooling1D(pool_size=(2,2), strides=4))
    model.add(Activation('relu'))

    #flatten & linear layers
    model.add(Flatten())
    model.add(Dense(42))
    model.add(Activation('relu'))

    #Classify if there is a merger waveform, output probability
    model.add(Dense(2, activation="softmax"))

    return model